In [2]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import json

from utils import *
from train import *
from evaluation import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')



<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\WattCast


In [ ]:
def train_eval_tuning():

    wandb.init(project="WattCast_tuning")
    wandb.config.update(config_run)
    config = wandb.config

    print("Getting data...")

    pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

    print("Getting model instance...")
    model_instance = get_model(config)
    model_instance, _ = train_models([model_instance], ts_train_piped, ts_train_weather_piped, ts_val_piped, ts_val_weather_piped)

    print("Evaluating model...")
    predictions, score = predict_testset(model_instance[0], 
                                  ts_test_piped[config.longest_ts_test_idx], 
                                  ts_test_weather_piped[config.longest_ts_test_idx],
                                  config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                  )


    print("Plotting predictions...")
    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'eval_loss': score})
    wandb.log({'predictions': fig})
    wandb.finish()



### Running

In [ ]:
locations, _ = get_hdf_keys(dir_path)
list(((list(locations.keys())[0].split(".")[0], location) for location in locations['2_town.h5']))

In [ ]:
#we keep the number of sweeps constant to ensure a fair playing field for all algorithms
sweeps = 10 # number of sweeps per model, per location: 

scale_location_pairs = [
('2_town', 'GLENDOVEER-13596'),
# ('2_town.h5', 'GLENDOVEER-13597'),
 #('2_town.h5', 'GLENDOVEER-13598'),
 #('2_town.h5', 'GLENDOVEER-13599'),
 #('2_town.h5', 'GLENDOVEER-CLIFFGATE'),
 #('2_town.h5', 'GLENDOVEER-NORTHEAST'),
 #('2_town', 'KELLY'),
 #('2_town', 'LENTS-13101'),
 #('2_town.h5', 'LENTS-HAPPY'),
 #('2_town.h5', 'LENTS-MT'),
 #('2_town.h5', 'LENTS-NORTH'),
 #('2_town', 'MIDWAY-DIVISION'),
 #('2_town.h5', 'MIDWAY-DOUGLAS'),
 #('2_town.h5', 'MIDWAY-LYNCH'),
 #('2_town.h5', 'MIDWAY-POWELLHURST'),
 #('2_town', 'RAMAPO-EMERALD'),
 #('2_town.h5', 'RAMAPO-GILBERT'),
 #('2_town.h5', 'RAMAPO-RAMAPO')
 ]

models = [
        #'rf',
        #'xgb', 
        #'gru', 
        #'lgbm',  
        'nbeats',
        #'tft'
        ]


for scale, location in scale_location_pairs:

    for model in models:
        # placeholder initialization of config file (will be updated in train_eval_light())
        config_run = {
            'spatial_scale': scale,
            'temp_resolution': 60,
            'location': location,
            'model': model,
            'horizon_in_hours': 24,
            'lookback_in_hours': 24,
            'boxcox': True,
            'liklihood': None,
            'weather': True,
            'holiday': True,
            'datetime_encodings': False,
            'heat_wave_binary': False,
        }
        
        with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
            sweep_config = json.load(fp)                  

        sweep_config['name'] = model + 'sweep' + config_run['spatial_scale'] + '_' + config_run['location'] + '_' + str(config_run['temp_resolution'])

        sweep_id = wandb.sweep(sweep_config, project="Portland_AMI_tuning")
        wandb.agent(sweep_id, train_eval_tuning, count=sweeps)
        wandb.finish()



### Degbugging

In [3]:
plot_location_splits(dir_path)